In [0]:
# Novo Notebook (ex: 03_tft_data_analysis.ipynb) - Célula 1: Configuração SparkSession

from pyspark.sql import SparkSession

print("Iniciando SparkSession para novo notebook...")

# A SparkSession já é fornecida e configurada pelo Databricks.
# Ela já herdará o acesso ao S3 via IAM Role do seu CloudFormation.
spark = SparkSession.builder.appName("TFTDataAnalysis").getOrCreate()

print("SparkSession configurada.")

In [0]:
# Novo Notebook - Célula 2: Consultando a tabela do Unity Catalog

# Nome completo da sua tabela no Unity Catalog
UNITY_CATALOG_TABLE_NAME = "tft_analytics.raw.tft_matches_raw"

print(f"Lendo dados da tabela '{UNITY_CATALOG_TABLE_NAME}'...")

# Opção 1: Usar Spark SQL (recomendado para consultas diretas)
df_raw_from_catalog_sql = spark.sql(f"SELECT * FROM {UNITY_CATALOG_TABLE_NAME}")
print("\nDados da tabela via Spark SQL (primeiras 5 linhas):")
df_raw_from_catalog_sql.show(5, truncate=False)

# Opção 2: Carregar como DataFrame PySpark (para manipulação posterior com Python)
df_raw_from_catalog_pyspark = spark.read.table(UNITY_CATALOG_TABLE_NAME)
print("\nDados da tabela via PySpark (esquema):")
df_raw_from_catalog_pyspark.printSchema()

print(f"\nTotal de registros na tabela '{UNITY_CATALOG_TABLE_NAME}': {df_raw_from_catalog_pyspark.count()}")

In [0]:
# Célula para Criar o Esquema 'curated'

CATALOG_NAME = "tft_analytics"
CURATED_SCHEMA_NAME = "curated"

print(f"Verificando e criando o esquema '{CATALOG_NAME}.{CURATED_SCHEMA_NAME}'...")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{CURATED_SCHEMA_NAME}").show()

print(f"Esquema '{CATALOG_NAME}.{CURATED_SCHEMA_NAME}' criado ou já existente.")

# Opcional: Conceder permissões se necessário (geralmente você já tem como criador)
# spark.sql(f"GRANT CREATE TABLE ON SCHEMA {CATALOG_NAME}.{CURATED_SCHEMA_NAME} TO `users`").show()
# spark.sql(f"GRANT USE SCHEMA ON SCHEMA {CATALOG_NAME}.{CURATED_SCHEMA_NAME} TO `users`").show()

In [0]:
# Célula para Criar uma View SQL
# Lembre-se que o nome da sua tabela é catalog.schema.table
UNITY_CATALOG_TABLE_NAME = "tft_analytics.raw.tft_matches_raw"
VIEW_NAME = "tft_analytics.curated.match_summary_view" # Nome da sua nova view

print(f"Criando a view '{VIEW_NAME}' a partir da tabela '{UNITY_CATALOG_TABLE_NAME}'...")

spark.sql(f"""
    CREATE OR REPLACE VIEW {VIEW_NAME} AS
    SELECT
        metadata.match_id,
        info.game_version,
        info.game_length,
        info.queue_id,
        info.tft_set_number
    FROM {UNITY_CATALOG_TABLE_NAME}
""").show(truncate=False) # .show() aqui é apenas para exibir o status da operação, não os dados da view

print(f"\nView '{VIEW_NAME}' criada com sucesso!")
print("Agora você pode consultar a view como se fosse uma tabela.")

# Testando a nova view
print(f"\nConsultando as primeiras 5 linhas da view '{VIEW_NAME}':")
spark.sql(f"SELECT * FROM {VIEW_NAME} LIMIT 5").show(truncate=False)

# Exemplo de consulta mais complexa na view
print(f"\nContando partidas por versão do jogo usando a view '{VIEW_NAME}':")
spark.sql(f"""
    SELECT
        game_version,
        COUNT(match_id) AS total_matches
    FROM {VIEW_NAME}
    GROUP BY game_version
    ORDER BY total_matches DESC
    LIMIT 5
""").show(truncate=False)